## 2-1. 제목 (Markdown)

In [1]:
# Assignment 5 — Model Evaluation (KoBART Summarization)

# 이 노트북은 Assignment 5에서 학습한 KoBART 한국어 요약 모델을
# Test set에서 평가하고, ROUGE 지표 및 예시 요약 결과를 확인하는 노트북입니다.

## 2-2. 환경 설정 & 경로 지정

In [2]:
!pip install -q transformers datasets sentencepiece accelerate evaluate rouge-score

import os
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
import evaluate

from google.colab import drive
drive.mount("/content/drive")

# ✅ 학습 때 저장한 모델 경로와 동일하게 설정할 것!
MODEL_DIR = "/content/drive/MyDrive/boncahier/models/kobart_ko_news"
print("MODEL_DIR:", MODEL_DIR)  # 또는 Google Drive/HF Hub 경로로 수정
SEED = 42

MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 128

# 평가 시 샘플 수 (None이면 전체)
MAX_TEST_SAMPLES = 1000

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
Mounted at /content/drive
MODEL_DIR: /content/drive/MyDrive/boncahier/models/kobart_ko_news


## 2-3. 데이터 로드 & 동일 분할


In [4]:
# ============================================================
# 1. Assignment 4에서 저장한 CSV를 이용해 test set 재구성
#    - training.ipynb와 동일한 방식으로 split해야 함 (seed 42, 8:1:1)
# ============================================================

csv_path = "data/naver_news_summarization_ko.csv"

ko_df = pd.read_csv(csv_path)
ko_df = ko_df[["document", "summary"]].dropna().reset_index(drop=True)

raw_dataset = Dataset.from_pandas(ko_df, preserve_index=False)
raw_dataset = raw_dataset.shuffle(seed=SEED)

train_valid_test = raw_dataset.train_test_split(test_size=0.2, seed=SEED)
temp = train_valid_test["test"]
valid_test = temp.train_test_split(test_size=0.5, seed=SEED)

dataset_dict = DatasetDict({
    "train": train_valid_test["train"],
    "validation": valid_test["train"],
    "test": valid_test["test"],
})

test_dataset = dataset_dict["test"]
print("Full test size:", len(test_dataset))

def maybe_subsample(ds, max_samples, seed=SEED):
    if max_samples is None or len(ds) <= max_samples:
        return ds
    return ds.shuffle(seed=seed).select(range(max_samples))

test_dataset = maybe_subsample(test_dataset, MAX_TEST_SAMPLES)
print("Subsampled test size:", len(test_dataset))


Full test size: 2220
Subsampled test size: 1000


## 2-4. 토크나이저 & 전처리

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    inputs = examples["document"]
    targets = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_SOURCE_LENGTH,
        padding="max_length",
        truncation=True,
    )

    labels = tokenizer(
        targets,
        max_length=MAX_TARGET_LENGTH,
        padding="max_length",
        truncation=True,
    )["input_ids"]

    labels = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_seq]
        for label_seq in labels
    ]
    model_inputs["labels"] = labels
    return model_inputs

tokenized_test = test_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=test_dataset.column_names,
)
tokenized_test[0]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'input_ids': [19250,
  248,
  14413,
  17076,
  28832,
  22729,
  14476,
  21982,
  19329,
  19610,
  14442,
  14048,
  19441,
  18555,
  29874,
  12024,
  19250,
  248,
  14413,
  17076,
  28832,
  298,
  13468,
  15872,
  29874,
  14307,
  17389,
  11699,
  29708,
  11821,
  14680,
  18555,
  29874,
  12005,
  19250,
  248,
  14413,
  17076,
  15326,
  15155,
  22729,
  14058,
  14182,
  250,
  15074,
  14129,
  27450,
  25438,
  14379,
  16259,
  12024,
  27331,
  12273,
  10487,
  12007,
  16890,
  16608,
  24374,
  10524,
  12007,
  14623,
  14781,
  16601,
  14253,
  14130,
  27331,
  12273,
  10487,
  12005,
  14641,
  21901,
  16495,
  14232,
  17581,
  14488,
  9499,
  16975,
  14053,
  10839,
  12037,
  14025,
  22676,
  13125,
  18658,
  21663,
  18188,
  15979,
  29262,
  1700,
  321,
  14879,
  310,
  14524,
  16452,
  14374,
  14696,
  14363,
  14370,
  27656,
  19294,
  14979,
  26392,
  14182,
  252,
  14515,
  14129,
  14650,
  16890,
  14049,
  14374,
  18725,
  1443

## 2-5. 모델 로드 & 평가

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

def compute_metrics(eval_pred):
    preds, labels = eval_pred

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 2) for k, v in result.items()}
    return result

eval_args = Seq2SeqTrainingArguments(
    output_dir="./eval_tmp",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=eval_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

metrics = trainer.evaluate(eval_dataset=tokenized_test)
metrics

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/tmp/ipython-input-813368061.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hhj2000 (hhj2000-hanguk-university-of-foreign-studies) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 0.4874518811702728,
 'eval_model_preparation_time': 0.0028,
 'eval_rouge1': 35.98,
 'eval_rouge2': 15.41,
 'eval_rougeL': 35.57,
 'eval_rougeLsum': 35.52,
 'eval_runtime': 73.8373,
 'eval_samples_per_second': 13.543,
 'eval_steps_per_second': 3.386}

## 2-6. 예시 출력 몇 개 보기

In [7]:
# Test set에서 몇 개 샘플 골라서 실제 요약 결과 확인
num_examples = 3
sample_indices = list(range(min(num_examples, len(test_dataset))))

for idx in sample_indices:
    example = test_dataset[idx]
    input_text = example["document"]
    ref_summary = example["summary"]

    inputs = tokenizer(
        input_text,
        max_length=MAX_SOURCE_LENGTH,
        truncation=True,
        return_tensors="pt",
    )

    # ✅ KoBART/BART는 token_type_ids 안 씀 → 제거
    inputs.pop("token_type_ids", None)

    # ✅ GPU 쓰는 경우 device로 이동(선택이지만 권장)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_length=MAX_TARGET_LENGTH,
            num_beams=6,          # ✅ 4 → 6
            length_penalty=1.0,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
    pred_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    print("="*80)
    print(f"[예시 #{idx}]")
    print("[원문 일부]\n", input_text[:400], "...\n")
    print("[실제 요약]\n", ref_summary, "\n")
    print("[모델 요약]\n", pred_summary, "\n")

[예시 #0]
[원문 일부]
 2021 기업시민보고서 발간…재생에너지 100% 사용 등 담아 포스코케미칼의 2021 기업시민보고서ⓒ포스코케미칼 데일리안 조인영 기자 포스코케미칼은 2021 기업시민보고서를 발간하고 2035년까지 배터리소재 사업 부문의 탄소중립을 달성하겠다는 로드맵을 공개했다고 6일 밝혔다. 탄소중립은 다양한 감축 활동을 통해 지구 온난화의 주범인 이산화탄소의 실질적인 배출량을 제로 zero 로 만드는 것을 의미하며 세계 각국 정부와 주요 기업들은 2050년까지 이를 달성하는 것을 목표로 하고 있다. 포스코케미칼은 2035년까지 배터리소재 사업 부문에서 탄소중립을 달성할 계획이다. 이는 배터리소재 선도 기업으로서 기후변화 문제 해결에 적극 나서고 주요 자동차사와 배터리 고객사들의 친환경적인 소재 생산 요구에 선제적으로 대응 ...

[실제 요약]
 포스코케미칼은 2021 기업시민보고서를 발간하고 2035년까지 배터리소재 사업 부문의 탄소중립을 달성하겠다는 로드맵을 공개했다고 6일 밝혔는데, 이는 배터리 선도 기업으로서 기후변화 문제 해결에 적극 나서고 주요 자동차사와 배터리 고객사들의 친환경적인 소재 생산 요구에 선제적으로 대응해 ESG 경쟁력을 확보하기 위함이며 포스코케미칼은 국내외 양·음극재 공장에 태양광 발전설비 도입을 확대하고 있으며 북미에 설립하는 GM과의 양극재 합작사 ‘얼티엄캠’에서도 수력 등의 재생에너지를 적극 활용할 계획이다. 

[모델 요약]
 포스코케미칼은 2021 기업시민보고서를 발간하고 2035년까지 배터리소재 사업 부문의 탄소중립을 달성하겠다는 로드맵을 공개했다고 6일 밝혔으며 이 로드맵은 다양한 감축 활동을 통해 지구 온난화의 주범인 이산화탄소의 실질적인 배출량을 제로 zero 로 만드는 것을 의미하며 세계 각국 정부와 주요 기업들은 2050년까지 이를 달성하는 것을 목표로 하고 있다. 

[예시 #1]
[원문 일부]
 LG엔솔 3% 넘게 하락중… 이대로면 신저가 경신 美 단독공장 투자 재검토에 보호예수 해제 겹쳐 2분